In [1]:
%%bash
OUT_DIR=/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/0X_contigAnnotation

makeblastdb -in $OUT_DIR/aclame_proteins_viruses_prophages_0.4.fasta -parse_seqids -dbtype prot -out $OUT_DIR/aclame_proteins_viruses_prophages_0.4



Building a new DB, current time: 07/20/2018 09:04:35
New DB name:   /ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/0X_contigAnnotation/aclame_proteins_viruses_prophages_0.4
New DB title:  /ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/0X_contigAnnotation/aclame_proteins_viruses_prophages_0.4.fasta
Sequence type: Protein
Keep Linkouts: T
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 54218 sequences in 1.94514 seconds.


In [6]:
%%writefile runBlastGenesVsALAME.sh
#!/bin/bash
#$ -N blastACLAME
#$ -pe parallel 6
#$ -l h_vmem=4G
#$ -l h_rt=72:00:00
#$ -cwd
#$ -m ea
#$ -M jmoreno@tuebingen.mpg.de

IN_DIR=/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2
OUT_DIR=/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/0X_contigAnnotation

blastp -db $OUT_DIR/aclame_proteins_viruses_prophages_0.4 -query $IN_DIR/00_Genes/Combined_InteMAP_pp_contigs.fa_LargerThan500.faa_USEDForDiamond -outfmt "6 qseqid pident saccver stitle staxids qlen slen qstart qend sstart send length bitscore gaps evalue" -out contigGenesVsACLAME.tsv -evalue 1e-5 -num_threads 6 

Overwriting runBlastGenesVsALAME.sh


In [4]:
%%writefile runBlastGenesVsNR.sh
#!/bin/bash
#$ -N blastNR
#$ -pe parallel 6
#$ -l h_vmem=4G
#$ -l h_rt=72:00:00
#$ -cwd
#$ -m ea
#$ -M jmoreno@tuebingen.mpg.de

IN_DIR=/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2
OUT_DIR=/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/0X_contigAnnotation

blastp -db /ebio/abt3_projects/databases/NCBI_blastdb/nr -query $IN_DIR/00_Genes/Combined_InteMAP_pp_contigs.fa_LargerThan500.faa_USEDForDiamond -outfmt "6 qseqid pident saccver stitle staxids qlen slen qstart qend sstart send length bitscore gaps evalue" -out contigGenesVsNR.tsv -evalue 1e-5 -num_threads 6 

Writing runBlastGenesVsNR.sh


In [5]:
%%bash
qsub runBlastGenesVsALAME.sh
qsub runBlastGenesVsNR.sh

Your job 1665855 ("blastACLAME") has been submitted
Your job 1665856 ("blastNR") has been submitted


In [36]:
%%writefile runDIAMONDGenesVsVIRAL.sh
#!/bin/bash
#$ -N diamondVIRAL
#-pe parallel 6
#$ -l h_vmem=8G
#$ -l h_rt=72:00:00
#$ -cwd
#$ -m ea
#$ -M jmoreno@tuebingen.mpg.de

IN_DIR=/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2
OUT_DIR=/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/0X_contigAnnotation

#/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/Programs/diamond/diamond makedb --in $IN_DIR/0X_contigAnnotation/ncbiViralDatabases/viral_ncbi.faa -d $OUT_DIR/ncbiViralDatabases/viral
#echo "database created..."

/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/Programs/diamond/diamond blastp --threads 6 -d $OUT_DIR/ncbiViralDatabases/viral -q $IN_DIR/00_Genes/Combined_InteMAP_pp_contigs.fa_LargerThan500.faa_USEDForDiamond -o $OUT_DIR/contigGenesVsVIRAL.out6.3columns.diamond --outfmt 6 qseqid stitle bitscore --max-target-seqs 25 --evalue 1e-5 

Overwriting runDIAMONDGenesVsVIRAL.sh


In [39]:
%%writefile runDIAMONDGenesVsNR.sh
#!/bin/bash
#$ -N diamondNR
#$ -pe parallel 50
#$ -l h_vmem=8G
#$ -l h_rt=72:00:00
#$ -cwd
#$ -m ea
#$ -M jmoreno@tuebingen.mpg.de

IN_DIR=/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2
OUT_DIR=/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp2/0X_contigAnnotation

#/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/Programs/diamond/diamond makedb --in $IN_DIR/0X_contigAnnotation/ncbiNRdatabase/nr -d $OUT_DIR/ncbiNRdatabase/nrDIAMOND
#echo "database created..."

#/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/Programs/diamond/diamond blastp --threads 50 -d $OUT_DIR/ncbiNRdatabase/nrDIAMOND -q $IN_DIR/00_Genes/Combined_InteMAP_pp_contigs.fa_LargerThan500.faa_USEDForDiamond -o $OUT_DIR/contigGenesVsNR.diamond --outfmt 100 --max-target-seqs 25 --evalue 1e-5 
/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/Programs/diamond/diamond blastp --threads 50 -d $OUT_DIR/ncbiNRdatabase/nrDIAMOND -q $IN_DIR/00_Genes/Combined_InteMAP_pp_contigs.fa_LargerThan500.faa_USEDForDiamond -o $OUT_DIR/contigGenesVsNR.diamond --outfmt 6 qseqid stitle bitscore --max-target-seqs 25 --evalue 1e-5 

Overwriting runDIAMONDGenesVsNR.sh


In [40]:
%%bash
#qsub runDIAMONDGenesVsVIRAL.sh
qsub runDIAMONDGenesVsNR.sh

Your job 1746680 ("diamondNR") has been submitted


### 
* At the end I used the diamond output (daa format) agains viralNR. 
* That file was loaded to MEGAN and taxonomy assignation of each protein was done with the LCA approach Parameters: Min Support: 1, Min Score: 40.0, Max Expected: 0.01, Top Percent: 10.0, Min-Complexity filter: 0.44.
* MEGAN's output was upload to the tmp2 folder as "contigGenesVsVIRALtax.MeganMapping.csv"